In [10]:
import whisper
# from pytube import YouTube
import os
from pytubefix import YouTube
from pytubefix.cli import on_progress
import json


def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            print(f"--- Directory created successfully: {directory} ---")
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

def youtube_video_download(url,main_directory):
    yt = YouTube(url, on_progress_callback = on_progress)
     
    ys = yt.streams.get_highest_resolution()

    vid_title=yt.title.replace(" ","_")
    output_path=f"{main_directory}/{vid_title}/"
    createDirectory(output_path)
    filename=f"youtube_video.mp4"
    
    file_path = os.path.join(output_path, filename)
    
    if os.path.isfile(file_path):
        print(f"--- Video already exists")
        pass
    else: 
        print(f"--- Download ... {yt.title} ---")
        ys.download(output_path=output_path, filename=filename)
        print(f"--- Video Download Done ---")
    
    return output_path, filename

def stt_with_whisper(directory, video_path):
    print("--- Whisper Model load ---")
    whisper_model = whisper.load_model("turbo")
    # transcription = whisper_model.transcribe(f"./{yt.title}.mp4", fp16 = False)["text"].strip()
    print("--- Start Transcribe ---")
    path=directory+video_path
    stt_result=whisper_model.transcribe(path, fp16 = False)

    return stt_result

def process_stt_by_minute(stt_data):
    print("--- segment stt result ---")
    result = []
    
    segments = stt_data['segments']
    
    if not segments:
        return result
        
    current_minute = 0
    current_texts = []
    current_start = 0
    
    for segment in segments:
        segment_start_minute = int(segment['start'] // 60)
        
        if segment_start_minute == current_minute:
            current_texts.append(segment['text'].strip())
        else:
            if current_texts:
                result.append({
                    "start": current_start,
                    "end": (current_minute + 1) * 60,
                    "text": " ".join(current_texts)
                })
            
            while current_minute + 1 < segment_start_minute:
                current_minute += 1
                result.append({
                    "start": current_minute * 60,
                    "end": (current_minute + 1) * 60,
                    "text": ""
                })
            
            current_minute = segment_start_minute
            current_texts = [segment['text'].strip()]
            current_start = current_minute * 60
    
    if current_texts:
        result.append({
            "start": current_start,
            "end": (current_minute + 1) * 60,
            "text": " ".join(current_texts)
        })
    
    return result

def save_captions_to_file(directory,result):
    stt_output_path=f"{directory}/transcription.json"
    with open(stt_output_path, 'w', encoding='utf-8') as outfile:
        json.dump(result, outfile,indent="\t",ensure_ascii=False)

In [11]:
url="https://youtu.be/95vT5rBiEYI?si=ruI2oOOIQLib8pDj" 
main_directory="/workspace/.gen/VLM/Video_to_chat/assets"
directory, video_path=youtube_video_download(url,main_directory)
stt_result=stt_with_whisper(directory, video_path)
minute_res=process_stt_by_minute(stt_result)
save_captions_to_file(directory,minute_res)

--- Video already exists
--- Whisper Model load ---
--- Start Transcribe ---
--- segment stt result ---


In [12]:
minute_res=process_stt_by_minute(stt_result)

--- segment stt result ---


In [13]:
len(minute_res)

17

In [14]:
minute_res

[{'start': 0,
  'end': 60,
  'text': '아이고! 아이고! 아이고! 아이고! 야, 이 드랙랑! 예! 야, 이거만이야! 아, 참! 아... 아... 왜, 왜? 오랜만이라... 어서오세요! 아우! 아우, GD! 나... 박수라도 한 번! 괜찮아요? 네, 아이... 나 반가울 거야! 많이 반가울 거야! 네... 네, 이제... G-Dragon! 아, 아! 아, 우리 또 GD 진짜 오랜만이네요! GD는 첫 출연이니까!'},
 {'start': 60,
  'end': 120,
  'text': '그렇죠! 그죠, 그죠? 네! 네... 진짜로... 제가... 이런... 토크? 해야라고 해야 되나요? 네, 네! 한 10년 만 정도라서... 맞아, 맞아! 지금 얘기 들어보니까 거의 힐링캠프가 마지막이었다고! 네... 네, 그죠? 오늘도 무슨 캠프로 온 것 같긴 한데... 그렇죠? 네... 네... 자, 제가 소활드릴게요! 영원한 건 절대 없어를 외쳤지만 영원한 시대 아이콘 우리 GDragon! 네! 권영수입니다! M swims Wild'},
 {'start': 120,
  'end': 180,
  'text': '워킹! 락아 락아! 락아! 사랑 같은 소리 따위 지민자 오늘 밤은 빛나가게 세상을 흔들어 세상을 흔들어 오랜만에 인사를 좀 해주세요. 네 안녕하세요. 이거 제가 왜 하겠... 네 안녕하세요. 가수 지드래곤입니다. 올해가 이제 값진 년이죠? 올해가 얼마 안 남긴 했습니다만 푸른 용의 해입니다. 지옹의 해를 맞아서 드디어 7년 만에! 이게 7년 만이네. 솔로 앨범이! 진짜 많은 분들이 지디가 또 어떤 곡을 발표할까 어떤 곡을 들고 나올까 관심도 굉장히 컸었고 한편으로 부담도 좀 될 것 같아. 지디 입장에서도 부담을 가진 지는 너무'},
 {'start': 180,
  'end': 240,
  'text': '오래됐다고 해야 되나 이게 매년 사실 데뷔 초 때부터 다음 앨범에는 더 잘해야 되겠다 이렇게 갔다 보니까 사실 지금은 부담이 기본적으로는 

# Description 확인

In [5]:
def extract_hashtags(text):
    words = text.replace('\n', ' ').split()
    
    hashtags = [word.strip() for word in words if word.startswith('#')]
    
    return hashtags

In [6]:
extract_hashtags(yt.description)

['#유퀴즈온더블럭', '#유퀴즈', '#유재석', '#조세호', '#티빙에서스트리밍']

In [18]:
a={70: {'start': 0,
  'end': 70,
  'text': 'll'},
 120: {'start': 60,
  'end': 120,
  'text': 'ss'},
 100: {'start': 120,
  'end': 100,
  'text': 'ssss'},
  }
b={item['end']: item for item in minute_res}

In [20]:
set(a.keys()) | set(b.keys())

{60,
 70,
 100,
 120,
 180,
 240,
 300,
 360,
 420,
 480,
 540,
 600,
 660,
 720,
 780,
 840,
 900,
 960,
 1020}

In [25]:
print(a.get(3,None))

None
